In [9]:
import pandas as pd
import plotly.graph_objects as go
#file_path = '/media/marc/Seagate Backup Plus Drive/all_veg_override_rosbag_grp_4/trajectory/17_loam_trajectory.bag_voxel_map_0.csv'
file_path = '/mnt/6C20E58F769B49C6/Users/Marc/Desktop/all_veg_override_rosbag_grp_4/11_loam_trajectory/voxel_map_0.csv'

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# Read the first 12 columns of the CSV file and ignore the rest
df = pd.read_csv(file_path, delimiter=',', usecols=range(12), header=None, skiprows=1)

# Define the columns manually
columns = ['VoxelX', 'VoxelY', 'VoxelZ', 'NumHits', 'NumMisses', 'MeanIntensity', 'VarianceIntensity', 'MeanReflectivity', 'VarianceReflectivity', 'Force', 'N_occ', 'l_occ']
df.columns = columns

# Convert numeric columns to appropriate types
numeric_columns = ['VoxelX', 'VoxelY', 'VoxelZ', 'NumHits', 'NumMisses', 'MeanIntensity', 'VarianceIntensity', 'MeanReflectivity', 'VarianceReflectivity', 'Force', 'N_occ', 'l_occ']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Display the first few rows and description of the dataframe
print("Initial dataframe:")
print(df.head())

print("\nDataframe description:")
print(df.describe())

print("\nCheck for NaN values in 'NumHits' column:")
print(df['NumHits'].isna().sum())

df_filtered = df[df['NumHits'] > 0]
print("\nFiltered dataframe (NumHits > 0):")
print(df_filtered)

print("\nNumber of entries where NumHits > 0:", len(df_filtered))

df = df_filtered

df['VoxelX'] = df['VoxelX'] * 0.1
df['VoxelY'] = df['VoxelY'] * 0.1
df['VoxelZ'] = df['VoxelZ'] * 0.1

voxel_size = .1


df['porosity'] = df['NumHits'] / (df['NumHits'] + df['NumMisses'])

max_reflectivity = df['MeanReflectivity'].max()
df['normalized_reflectivity'] = df['MeanReflectivity'] / max_reflectivity

max_intensity = df['MeanIntensity'].max()
df['normalized_intensity'] = df['MeanIntensity'] / max_intensity

max_force = df['Force'].max()
df['normalized_force'] = df['Force'] / max_force


##
## Log-transform the force values
##
df['log_force'] = np.log1p(df['Force'])  # log1p is used to avoid log(0) issues
##
##
##


# Normalize the log-transformed force values
max_log_force = df['log_force'].max()
df['normalized_force'] = df['log_force'] / max_log_force


# Optionally print statistics about force
print("max force: ", max_force)
print("min force: ", df['Force'].min())
print("mean force: ", df['Force'].mean())


mean_porosity = df['porosity'].mean()
median_porosity = df['porosity'].median()
min_porosity = df['porosity'].min()
max_porosity = df['porosity'].max()
print("max porosity: ", df['porosity'].max())
print("min porosity: ", df['porosity'].min())
print("mean porosity: ", df['porosity'].mean())
# Set the porosity of the top X voxels to the next highest value
#if X < len(df):
#    next_highest_porosity = df.iloc[X]['porosity']
#    df.iloc[:X, df.columns.get_loc('porosity')] = next_highest_porosity

# Recalculate the maximum porosity after adjustment
max_porosity = df['porosity'].max()
df = df.sort_index()


cuboids = []
df['inverted_reflectivity'] = 1 - df['normalized_reflectivity']

# Iterate through each voxel and create its cuboid
for idx, row in df.iterrows():
    x, y, z = row['VoxelX'], row['VoxelY'], row['VoxelZ']
    normalized_force = row['normalized_force']
    normalized_reflectivity = row['normalized_reflectivity']
    normalized_intensity = row['normalized_intensity']
    
    inverted_reflectivity = row['inverted_reflectivity']
    
    # Create a cuboid (box) mesh
    mesh_box = o3d.geometry.TriangleMesh.create_box(width=voxel_size, height=voxel_size, depth=voxel_size)
    mesh_box.translate((x - voxel_size / 2, y - voxel_size / 2, z - voxel_size / 2))
    
    # Color the cuboid based on the normalized force
    color = plt.cm.viridis(normalized_force)[:3]  # Normalize and convert to RGB
    mesh_box.paint_uniform_color(color)
    
    # Add the cuboid to the list
    cuboids.append(mesh_box)

all_cuboids = cuboids[0]
for cuboid in cuboids[1:]:
    all_cuboids += cuboid

o3d.visualization.draw_geometries([all_cuboids])
    

max force:  0.242498
min force:  -0.00768821
mean force:  0.12894758316241198
max porosity:  1.0
min porosity:  7.620208793720948e-05
mean porosity:  0.47595491692159614


In [1]:
import pandas as pd
import plotly.graph_objects as go
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# Adjusted to read the entire CSV, including R, G, B columns
file_path = '/mnt/6C20E58F769B49C6/Users/Marc/Desktop/all_veg_override_rosbag_grp_4/13_loam_trajectory/voxel_map_100.csv'

# Read the entire CSV file including the RGB columns
df = pd.read_csv(file_path, delimiter=',', usecols=range(15), header=None, skiprows=1)

# Define the columns manually including RGB
columns = ['VoxelX', 'VoxelY', 'VoxelZ', 'NumHits', 'NumMisses', 'MeanIntensity', 'VarianceIntensity', 'MeanReflectivity', 'VarianceReflectivity', 'Force', 'N_occ', 'l_occ', 'R', 'G', 'B']
df.columns = columns

# Convert numeric columns to appropriate types
numeric_columns = ['VoxelX', 'VoxelY', 'VoxelZ', 'NumHits', 'NumMisses', 'MeanIntensity', 'VarianceIntensity', 'MeanReflectivity', 'VarianceReflectivity', 'Force', 'N_occ', 'l_occ', 'R', 'G', 'B']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Filter out rows with NaN values in important columns if necessary
df_filtered = df[df['NumHits'] > 0]
df = df_filtered

# Scale voxel coordinates to match the desired size
df['VoxelX'] = df['VoxelX'] * 0.1
df['VoxelY'] = df['VoxelY'] * 0.1
df['VoxelZ'] = df['VoxelZ'] * 0.1

voxel_size = .1

# Additional calculations if needed
df['porosity'] = df['NumHits'] / (df['NumHits'] + df['NumMisses'])
df['normalized_force'] = np.log1p(df['Force']) / np.log1p(df['Force']).max()

# Variable to control the visualization mode
show_non_zero_rgb = True  # Initially show voxels with non-zero RGB

# Iterate through each voxel and create its cuboid
cuboids_with_rgb = []
cuboids_with_force = []

for idx, row in df.iterrows():
    x, y, z = row['VoxelX'], row['VoxelY'], row['VoxelZ']
    normalized_force = row['normalized_force']

    r, g, b = row['R'], row['G'], row['B']
    if r > 0 or g > 0 or b > 0:
        color = [r, g, b]
        cuboids_with_rgb.append(o3d.geometry.TriangleMesh.create_box(width=voxel_size, height=voxel_size, depth=voxel_size))
        cuboids_with_rgb[-1].translate((x - voxel_size / 2, y - voxel_size / 2, z - voxel_size / 2))
        cuboids_with_rgb[-1].paint_uniform_color(color)
    
    # Include all voxels with force for the second window
    color = plt.cm.viridis(normalized_force)[:3]
    cuboids_with_force.append(o3d.geometry.TriangleMesh.create_box(width=voxel_size, height=voxel_size, depth=voxel_size))
    cuboids_with_force[-1].translate((x - voxel_size / 2, y - voxel_size / 2, z - voxel_size / 2))
    cuboids_with_force[-1].paint_uniform_color(color)

# Combine cuboids within each category
all_cuboids_with_rgb = cuboids_with_rgb[0] if cuboids_with_rgb else o3d.geometry.TriangleMesh()
for cuboid in cuboids_with_rgb[1:]:
    all_cuboids_with_rgb += cuboid

all_cuboids_with_force = cuboids_with_force[0] if cuboids_with_force else o3d.geometry.TriangleMesh()
for cuboid in cuboids_with_force[1:]:
    all_cuboids_with_force += cuboid

# Visualize in two separate windows
vis_with_rgb = o3d.visualization.Visualizer()
vis_with_rgb.create_window(window_name="Voxels with RGB")

vis_with_force = o3d.visualization.Visualizer()
vis_with_force.create_window(window_name="Voxels with Force (colored by force)")

# Add the geometries to the respective visualizers
vis_with_rgb.add_geometry(all_cuboids_with_rgb)
vis_with_force.add_geometry(all_cuboids_with_force)

# Run the visualizers in a non-blocking manner
while True:
    vis_with_rgb.update_geometry(all_cuboids_with_rgb)
    vis_with_force.update_geometry(all_cuboids_with_force)
    vis_with_rgb.poll_events()
    vis_with_force.poll_events()
    vis_with_rgb.update_renderer()
    vis_with_force.update_renderer()

    # Check for termination conditions using 'capture_screen_float_buffer'
    if not vis_with_rgb.capture_screen_float_buffer(do_render=False) or \
       not vis_with_force.capture_screen_float_buffer(do_render=False):
        break

# Destroy the visualizer windows
vis_with_rgb.destroy_window()
vis_with_force.destroy_window()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
